# Preprocessing

The preprocessing pipeline should output a list of speakers with quotes. Each element of this list should contain information about the speaker (full name, gender, date of birth...) - obtained from wikidata, and a string formed by joining multiple quotes in order to get 5000 characters of text (restriction of the LIWC model).

The pipeline is presented through a data analysis example - analysing the personalities of the US politicians. In this example we take 100 politicians from both of the two major political parties, the Democratic party and the Republican party. We select the politicians which have the most quotes in our database. We only consider quotes for which the probability of the speaker is higher than 80% (referenced as significant quotes in this notebook).

In [2]:
import bz2
import json
import re
import random
import sys
import os
import bz2
import time

### Counting significant quotes

<b>This step has been executed once and will not be needed in following analyses since the quote counts calculated are for all the speakers and the output can be simply reused.</b>

Define some methods for better reusability.

In [21]:
PATTERN_INPUT = "../quotebank/quotes-{}.json.bz2"

In [6]:
def write_json_to_file(name, obj):
    # Use current timestamp to make the name of the file unique
    millis = round(time.time() * 1000)
    name = f'{name}_{millis}.json'
    with open(name, 'wb') as f:
        output = json.dumps(obj)
        f.write(output.encode('utf-8'))
    return name

Methods used for counting significant quotes.

In [7]:
signi_count = 0
signi_quote_dict = {}

In [15]:
# The signature remains from an older version of the code, parameter out_file could be removed, but then has to be removed in other places in the code as well.
def initialize(out_file):
    global signi_count
    global signi_quote_dict
    signi_count = 0
    signi_quote_dict = {}

In [17]:
# The signature remains from an older version of the code, parameter out_file could be removed, but then has to be removed in other places in the code as well.
def count_significant_quotes(out_file, row):
    global signi_count
    global signi_quote_dict
    
    probas = row['probas']
    qids = row['qids']
    
    if (len(probas) == 0 or len(qids) == 0):
        return
    
    if (probas[0][0] == 'None'):
        return
    
    p = float(probas[0][1])
    if (p < 0.8):
        return
    
    qid = qids[0]
    
    signi_count = signi_count + 1
    signi_quote_dict[qid] = signi_quote_dict.get(qid, 0) + 1

General methods used for processing the quotes files.

In [5]:
"""
Process a chunk of the input stream.
"""
def proc(input, evaluate_quote, max_length=20):
    # Ugly global variable usage :(
    global index
    global invalid_json_count
    global invalid_chunk_count
    global chunk_stitching
    global stitch_length
    global scrap_next
    global quote_is_open
    global quote_part
    global dat_part
    global euro_error
    global euro_count
    
    global totin
    global totout
    global prev
    global dec
    global start
    """Decompress and process a piece of a compressed stream"""
    dat = dec.decompress(input)
    got = len(dat)
    if got != 0:    # 0 is common -- waiting for a bzip2 block
        try:
            if (euro_error):
                # If the previous chunk ended unexpectedly and could not be decoded, try to combine it with this chunk
                s = (dat_part + dat).decode('utf-8')
                euro_error = False
            else:
                # Decode the current chunk
                s = dat.decode('utf-8')
                
            # List elements in the quote files are separated by new lines (\n)
            lines = s.split('\n')

            for line in lines:
                try:
                    if (scrap_next):
                        # If the object spans too many chunks we decide to scrap it, and keep scraping until JSON can parse the line (chunk)
                        ob = json.loads(line)
                        scrap_next = False
                        quote_is_open = False
                        chunk_stitching -= stitch_length
                    else:
                        if (quote_is_open):
                            # If previous chunk ended in the middle of a JSON object we merge that content with the current line
                            ob = json.loads(quote_part + line)
                            quote_is_open = False
                        else:
                            # Parse the current line
                            ob = json.loads(line)

                    # Parametrization - do work on a single quote JSON object
                    evaluate_quote({}, ob)
                except ValueError:
                    """
                    Error occurs when the line does not contain the whole JSON object, which happens for the last line in almost every chunk of input stream.
                    We solve this by remembering the partial object, and then merging it with the rest of the object when we load the next chunk.
                    JSON object might span more than 2 chunks, and in that case we keep merging until we reach max_length chunks, when we just throw away the object
                    and count it as invalid using invalid_json_count.
                    """
                    if (scrap_next):
                        pass
                    else:
                        if (quote_is_open):
                            chunk_stitching += 1
                            quote_part = quote_part + line
                            stitch_length += 1

                            if (stitch_length > max_length):
                                invalid_json_count += 1
                                scrap_next = True
                        else:
                            quote_is_open = True
                            quote_part = line
                            stitch_length = 0
        except UnicodeDecodeError as e:
            # Error occurs when input stream is split in the middle of a character which is encoded with multiple bytes, for example the euro symbol
            if (euro_error):
                dat_part = dat_part + dat
            else:
                euro_error = True
                dat_part = dat
            
            euro_count += 1
        
        index += 1
    return got

In [24]:
def run_through_quotes(init, evaluate_quote, year, target_dict_name, path_to_input, name='test', chunk_size=16384):
    global index
    global invalid_json_count
    global invalid_chunk_count
    global chunk_stitching
    global stitch_length
    global scrap_next
    global quote_is_open
    global quote_part
    global dat_part
    global euro_error
    global euro_count
    
    global totin
    global totout
    global prev
    global dec
    global start
    
    size = os.path.getsize(path_to_input)
    invalid_json_count = 0
    invalid_chunk_count = 0
    chunk_stitching = 0
    stitch_length = 0
    scrap_next = False
    quote_is_open = False
    quote_part = ''
    dat_part = 0
    euro_error = False
    euro_count = 0
    
    totin = 0
    totout = 0
    prev = -1
    dec = bz2.BZ2Decompressor()
    start = time.time()
    
    init({})
    
    target_dict = poli_quotes if target_dict_name == "poli_quotes" else signi_quote_dict
    index = 0
    with open(path_to_input, 'rb') as f:
        for chunk in iter(lambda: f.read(chunk_size), b''):
            # feed chunk to decompressor
            got = proc(chunk, evaluate_quote)

            # handle case of concatenated bz2 streams
            if dec.eof:
                rem = dec.unused_data
                dec = bz2.BZ2Decompressor()
                got += proc(rem, evaluate_quote)

            # show progress
            totin += len(chunk)
            totout += got
            if got != 0:    # only if a bzip2 block emitted
                frac = round(1000 * totin / size)
                if frac != prev:
                    left = (size / totin - 1) * (time.time() - start)
                    print(f'\r{frac / 10:.1f}% (~{left:.1f}s left)\tyear: {year}\tnumber of speakers: {len(target_dict)}\tstitching: {chunk_stitching}\teuro count: {euro_count}\tinvalid json count: {invalid_json_count}\tinvalid chunk count: {invalid_chunk_count}', end='')
                    prev = frac

    # Show the resulting size.
    print(end='\r')
    print(totout, 'uncompressed bytes')

    output_name = write_json_to_file(f'{name}-{year}', target_dict)
    return output_name

Create files for every year, each file contains a dictionary where the key is the QID of the speaker, and the value is the number of significant quotes.
<br><br>
<font color='red'>WARNING: LONG EXECUTION!</font>

In [ ]:
years = [2015, 2016, 2017, 2018, 2019, 2020]
# years = [2020]
for year in years:
    path_to_input = PATTERN_INPUT.format(year)
    
    run_through_quotes(
        initialize, count_significant_quotes, year, "signi_quote_dict", path_to_input, name='signi-quote-count', chunk_size=1_048_576)
    print('')
    print(f'Finished compiling quotes for the year {year}')

Now combine the quote counts into a single file.
<br>
An example of the file names is used, the string should be updated if the code is ran again.

In [26]:
signi_quotes_file_names = [
    "signi-quote-count-2015_1636244638891.json",
    "signi-quote-count-2016_1636246832187.json",
    "signi-quote-count-2017_1636249273913.json",
    "signi-quote-count-2018_1636250518608.json",
    "signi-quote-count-2019_1636251729971.json",
    "signi-quote-count-2020_1636237785105.json"
]

In [ ]:
combined_signi_dict = {}

for file_name in signi_quotes_file_names:
    with open(file_name, 'r') as f:
        one_dict = json.load(f)
        for k in one_dict.keys():
            combined_signi_dict[k] = combined_signi_dict.get(k, 0) + one_dict[k]

Sort the dictionary so the speakers with the most quotes appear first.

In [ ]:
sorted_combined_signi_dict = {k: v for k, v in sorted(combined_signi_dict.items(), key=lambda item: item[1], reverse=True)}

And finally save the resulting dictionary into a file, this file can later be reused for multiple analyses, whenever we need to choose a representation of a group of people using the number of quotes to pick the most quoted individuals.

In [ ]:
write_json_to_file('signi-quote-count-combined', sorted_combined_signi_dict)

### Get the wikidata

Merge duplicate objects representing a single speaker but with differing fields.
<br>
Example: Arnold Schwarzenegger has both Austrian and American nationalities, and would appear twice, once with Austrian, and once with American nationality.

In [28]:
with open("../quotebank/american_politicians_fixed.json", "r") as f:
    wiki_poli = json.load(f)

In [ ]:
ampoli2 = dict()

index = 0
for row in wiki_poli:
    item = row['item']
    if item in ampoli2:
        tmp = ampoli2[item]
        columns = ['itemLabel', 'genderLabel', 'citizenshipLabel', 'religionLabel', 'ethnicLabel', 'degreeLabel', 'dateOfBirth', 'placeOfBirthLabel', 'memberOfParty', 'memberOfPartyLabel', 'languageLabel']
        for col in columns:
            if row.get(col, None) is None:
                continue
                
            t = tmp.get(col, None)
            
            if t is None:
                t = row[col]
            elif isinstance(t, list):
                if row[col] not in t:
                    t.append(row[col])
            elif row[col] != t:
                t = [t, row[col]]
                
            tmp[col] = t
    else:
        ampoli2[item] = row